In [1]:
# from SpiderMon import *
from matplotlib import pyplot as plt
from pprint import pprint
import os
import json
from collections import Counter, defaultdict

if not os.path.abspath('.').endswith('netscope'):
    root_path = os.path.dirname(os.path.dirname(os.path.abspath('.')))
    os.chdir(root_path)
    from analysis.load import Loader

from IntSight import *

In [2]:
def rater(exp_kind, answer, culprits, topo):
    ans = answer[exp_kind][0]
    index = 0
    gt = ans['groundTruth']
    
    if exp_kind == "burst":
        culprit_flows = sorted(culprits.value_counts(['flow']).items(), key=lambda x: x[1], reverse=True)
        # culprit_flows = sorted(culprits.value_counts(['e_bytes']).items(), key=lambda x: x[1], reverse=True)
        culprit_flows_list = [cf[0][0] for cf in culprit_flows]
        answer_flow = ans['src'] + "-" + ans['dst']
        if answer_flow in culprit_flows_list:
            index = culprit_flows_list.index(answer_flow) + 1
        
    elif exp_kind == "ecmp_imbalance":
        sw_contributors = defaultdict(float)
        for _, row in culprits.iterrows():
            src, dst = row.flow.split('-')
            paths = topo.get_shortest_paths_between_nodes(src, dst)
            for i, cp in enumerate(row.contention_points):
                if cp == '1':
                    for p in paths:
                        sw_contributors[p[i+1]] += 1/len(paths)
                        # sw_contributors.append(p[i+1]) # +1: src host
        sw_counter = sorted(sw_contributors.items(), key=lambda c: c[1], reverse=True)
        sw_counter = [sw[0] for sw in sw_counter]
        if gt in sw_counter:
            index = sw_counter.index(gt) + 1
    
    elif exp_kind == "delay":
        delay_culprits = culprits[culprits.high_delays > 0]
        links = []
        for _, row in delay_culprits.iterrows():
            path = row.whole_path.split(',')[1:-1] # without hosts
            for ih, hop in enumerate(row.contention_points):
                if hop == '1' and ih > 0:
                    ports.append(path[ih-1] + ',' + path[ih] + ',')
        port_counter = Counter(links)
        link_list = sorted(port_counter, key=lambda c: c[1], reverse=True)
        if gt in link_list:
            index = link_list.index(gt) + 1
    
    elif exp_kind == "port_queue":
        culprits
        ports = []
        for _, row in culprits.iterrows():
            path = row.whole_path.split(',')[1:-1] # without hosts
            for ih, hop in enumerate(row.contention_points):
                if hop == '1' and ih < len(path) - 1:
                    ports.append(path[ih] + ',' + path[ih+1] + ',')
        port_counter = Counter(ports)
        port_list = sorted(port_counter, key=lambda c: c[1], reverse=True)
        if gt in port_list:
            index = port_list.index(gt) + 1
    return index

In [12]:
# log_folder = "/home/user/dds/netscope/experiment/data/slot2/s2l4/burst/"
log_folder = "/mnt/netscope/data/slot2/s2l4/ecmp_imbalance/"
# log_folder = "/mnt/netscope/data/slot2/s2l4/burst2/"
# log_folder = "/mnt/netscope/data/slot2/fattree4/burst/"
# log_folder = "/mnt/netscope/data/slot2/fattree4/delay/"
# log_folder = "/mnt/netscope/data/slot2/fattree4/port_queue3/"
# log_folder = "/mnt/netscope/data/slot2/s2l4/ecmp_imbalance/"

exp_ids = os.listdir(log_folder)
# exp_ids = ['20221005_1859GMT']

rca_result = []
for ie, exp_id in enumerate(exp_ids):
    print(ie, exp_id, end="...")
    log_dir = os.path.join(log_folder, exp_id)
    
    with open(os.path.join(log_dir, 'answer.json'), 'r') as f:
        answer = json.load(f)
    exp_kind = list(answer.keys())[0]
    print(exp_kind, end="...")
    
    loader = Loader(log_dir)
    hosts = loader.load_hosts(debug=True)
    topo = loader.get_topo()
    
    
    reports = IntSight(hosts)
    
    culprits = reports[(reports.drops>0) 
                | (reports.high_delays > 0) 
                | (reports.contention_points.str.strip('-') != "") 
                | (reports.suspicion_points.str.strip('-') != "")]
    
    index = rater(exp_kind, answer, culprits, topo)
    
    print(index)
    
    rca_result.append(dict(
        id=exp_id,
        index=index,
        culprits=culprits,
        answer=answer,    
    ))
    # break

0 20221005_0935GMT...ecmp_imbalance...1
1 20221005_0951GMT...ecmp_imbalance...1
2 20221005_0955GMT...ecmp_imbalance...5
3 20221005_1000GMT...ecmp_imbalance...5
4 20221005_1042GMT...ecmp_imbalance...5
5 20221005_1047GMT...ecmp_imbalance...5
6 20221005_1052GMT...ecmp_imbalance...4
7 20221005_1054GMT...ecmp_imbalance...4
8 20221005_1109GMT...ecmp_imbalance...2
9 20221005_1131GMT...ecmp_imbalance...1
10 20221005_1133GMT...ecmp_imbalance...2
11 20221005_1146GMT...ecmp_imbalance...4
12 20221005_1148GMT...ecmp_imbalance...4
13 20221005_1153GMT...ecmp_imbalance...2
14 20221005_1156GMT...ecmp_imbalance...3
15 20221005_1200GMT...ecmp_imbalance...4
16 20221005_1203GMT...ecmp_imbalance...2
17 20221005_1210GMT...ecmp_imbalance...5
18 20221005_1213GMT...ecmp_imbalance...1
19 20221005_1225GMT...ecmp_imbalance...6
20 20221005_1227GMT...ecmp_imbalance...3
21 20221005_1237GMT...ecmp_imbalance...4
22 20221005_1239GMT...ecmp_imbalance...4
23 20221005_1244GMT...ecmp_imbalance...3


In [9]:
culprits.value_counts(['flow'])
reports.value_counts(['flow'])
# hosts.value_counts(['flow'])

flow 
h7-h4    113
h8-h2    101
h3-h5     91
h4-h7     84
h6-h1     67
h5-h1     38
h2-h3     37
h1-h4     34
h1-h3     21
dtype: int64

In [6]:
answer

{'burst': [{'src': 'h4',
   'dst': 'h8',
   'count': 544,
   'groundTruth': 's4-s6',
   'paths': ['s4,s1,s6,', 's4,s2,s6,'],
   'abnormalKind': 'flow'}]}

In [14]:
def topk(k, counter):
    return sum([counter.get(i, 0) for i in range(1, k+1)])
rca_counts = Counter([r['index'] for r in rca_result])
print(rca_counts)
L = len(rca_result)

for k in [1, 2, 3, 5, 10]:
    tk = topk(k, rca_counts)
    print(f"top{k}: {tk}/{L} ({tk/L*100:.1f}%)")
    if tk == L: break

Counter({4: 7, 5: 5, 1: 4, 2: 4, 3: 3, 6: 1})
top1: 4/24 (16.7%)
top2: 8/24 (33.3%)
top3: 11/24 (45.8%)
top5: 23/24 (95.8%)
top10: 24/24 (100.0%)


In [5]:
culprits

,i_epoch,e_epoch,flow,path_id,contention_points,suspicion_points,high_delays,delay,drops,i_bytes,e_bytes,e_pkts,whole_path
0,0,1,h1-h4,1,11-,---,1,67512,0,716,414,1,"h1,s3,s1,s4,h4"
1,4,5,h3-h6,4,111,---,1,100943,0,291,347,1,"h3,s4,s2,s5,h6"
2,5,6,h4-h5,0,111,---,1,100708,0,262,318,1,"h4,s4,s1,s5,h5"
3,12,13,h7-h5,5,1--,---,1,39928,0,288,344,1,"h7,s6,s2,s5,h5"
26,133,134,h7-h5,1,-1-,---,1,33597,0,288,344,1,"h7,s6,s1,s5,h5"
38,185,186,h1-h4,5,1--,---,1,78637,0,171,227,1,"h1,s3,s2,s4,h4"
39,187,188,h3-h6,0,-1-,---,1,28405,0,582,347,1,"h3,s4,s1,s5,h6"
40,197,198,h5-h7,4,--1,---,1,22101,0,672,392,1,"h5,s5,s2,s6,h7"
50,248,249,h1-h4,1,--1,---,1,75719,0,576,344,1,"h1,s3,s1,s4,h4"
56,278,279,h1-h4,1,--1,---,1,669844,0,459,227,1,"h1,s3,s1,s4,h4"


In [6]:
culprits[culprits.e_pkts > 1]

,i_epoch,e_epoch,flow,path_id,contention_points,suspicion_points,high_delays,delay,drops,i_bytes,e_bytes,e_pkts,whole_path


In [7]:
pd_code = '\tdict(name="IntSight", '
for k in [1, 2, 3, 5, 10]:
    tk = topk(k, rca_counts)
    pd_code += f"t{k}={tk}/L, "
    if tk == L: break
pd_code = pd_code.rstrip(", ")+"),"
print(pd_code)

	dict(name="IntSight", t1=0/L, t2=0/L, t3=0/L, t5=0/L, t10=0/L),


In [8]:
exam_score = 0
L_count = 0
for index, count in rca_counts.items():
    if index == 0: continue
    exam_score += (index - 1) * count
    L_count += count
print(exam_score, L_count, exam_score/L_count)

ZeroDivisionError: division by zero